In [ ]:
#  Import necessary libraries
!pip install jsonlines
!pip install peft
!pip install datasets
!pip install langchain-community
!pip install langchain
!pip install langchain_voyageai
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig,pipeline
import os
import pandas as pd
import jsonlines
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import TrainingArguments,Trainer
from peft import LoraModel, LoraConfig
from pprint import pprint

import datasets
from datasets import load_dataset
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_voyageai import VoyageAIEmbeddings

# from prompt_template import template

In [ ]:
# prompt: ModuleNotFoundError: No module named 'langchain_voyageai'

!pip install langchain_voyageai


### Loading Documents

In [ ]:
%pip install pymupdf
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import MathpixPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader

In [ ]:
loader = PyMuPDFLoader("60.pdf")
# loader = PyMuPDFLoader("sample_docs.pdf")
loader

In [ ]:
data = loader.load()

In [ ]:
import pandas as pd

# Create an empty DataFrame
empty_df = pd.DataFrame()
empty_df
empty_df['content_length'] = [len(page.page_content) for page in data]
empty_df.describe()

,content_length
count,29.000000
mean,1936.379310
std,570.038997
min,399.000000
25%,1749.000000
50%,2106.000000
75%,2274.000000
max,2611.000000


### Spliting

In [ ]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=250,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)


In [ ]:
splits = text_splitter.split_documents(data)
splits

[Document(page_content='ACT CIVIL & ADMINISTRATIVE TRIBUNAL \n \n \n \n \nLEE v GUO (Residential Tenancies) [2017] ACAT 60 \n \nRT 166/2017 \n \nCatchwords: \nRESIDENTIAL TENANCIES – compensation – whether there \nwas a breach of peace, comfort and privacy – whether the tenant', metadata={'source': '60.pdf', 'file_path': '60.pdf', 'page': 0, 'total_pages': 29, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word for Microsoft 365', 'producer': 'Microsoft® Word for Microsoft 365', 'creationDate': "D:20240329204618+05'30'", 'modDate': "D:20240329204618+05'30'", 'trapped': ''}),
 Document(page_content='needs to establish a ‘significant’ interference with the use of the \npremises – what ‘significant’ means in the context of section 71 \nof the RT Act – when considering whether there has been a', metadata={'source': '60.pdf', 'file_path': '60.pdf', 'page': 0, 'total_pages': 29, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': 

### Embedding and vectorStore

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
persist_directory = './chroma'

In [ ]:
embeddings = VoyageAIEmbeddings(
    voyage_api_key="pa-nDl_h1Mmg-73KP19sGFi4OAl9cyLiABswG7Rm6JnSZs", model="voyage-large-2"
)

batch size None


In [ ]:
%pip install chromadb
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory
)

ValueError: Expected EmbeddingFunction.__call__ to have the following signature: odict_keys(['self', 'input']), got odict_keys(['args', 'kwargs'])
Please see https://docs.trychroma.com/embeddings for details of the EmbeddingFunction interface.
Please note the recent change to the EmbeddingFunction interface: https://docs.trychroma.com/migration#migration-to-0416---november-7-2023 


In [ ]:
# prompt: ValueError                                Traceback (most recent call last)
# <ipython-input-15-ebe4e4552c4c> in <cell line: 2>()
#       1 get_ipython().run_line_magic('pip', 'install chromadb')
# ----> 2 vectordb = Chroma.from_documents(
#       3     documents=splits,
#       4     embedding=embeddings.embed_documents,
#       5     persist_directory=persist_directory
# 9 frames
# /usr/local/lib/python3.10/di

!pip install chromadb


60.pdf  chroma/  sample_data/


In [ ]:
vectordb.persist()

NameError: name 'vectordb' is not defined

### Loading Model

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
API_KEY= ""

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    top_k=1,
    top_p=0.9,
    temperature=0.1,

    huggingfacehub_api_token= API_KEY
)

#### Retrieval

In [ ]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """
[INST] You are chatbot who is specialized in Legal field.
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Helpful Answer: [/INST] """
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [ ]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
question1 = "What is the main theme discussed in this text?"
question2 = "What precedents are referred to in the text for further understanding of the theme?"
question3 = " Highlight the statutory aspects laid down in the texts."

In [ ]:
result = qa_chain({"query": question1})
print(result["result"])

In [ ]:
result = qa_chain({"query": question2})
print(result["result"])

In [ ]:
result = qa_chain({"query": question3})
print(result["result"])